# FOR TESTING: Remove before use

In [1]:
%env AIBS_RIG_ID = NP.0

env: AIBS_RIG_ID=NP.0


***
# **Without mouse on stage**
## Restart kernel before running this cell

In [2]:
# %run pip install --extra-index-url https://pypi.org/simple  --upgrade --quiet np_workflows
# %load_ext autoreload
# %autoreload 2

import datetime
import pathlib
import time

import IPython
import ipywidgets as widgets

import np_logging
import np_services
import np_session
import np_workflows
import np_workflows.npxc as npxc

from np_services import (
    Sync, VideoMVR, Cam3d, ScriptCamstim, NewScaleCoordinateRecorder, OpenEphys
)
from np_services.protocols import (
    Pretestable, Initializable, Startable, Testable, Stoppable, Finalizable, Pretestable, Verifiable, Validatable,
    TestError,
    )

import ttn_widgets

logger = np_logging.getLogger()
logger.setLevel('INFO')

np_workflows.elapsed_time_widget()

***
## "Quiet mode"
on  
*default*
- exception details are hidden
- regular messages displayed (log level = INFO)

off
- full exception details with traceback
- extra messages displayed (log level = DEBUG)

In [3]:
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

***
## Launch apps via RSC
[optional]

In [4]:
np_services.start_rsc_apps()

***
## Select mouse and user

In [5]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain for mouse
[optional]

In [6]:
np_workflows.mtrain_widget(mouse)

***
## Generate new session and folder 

In [7]:
#! for testing
session = np_session.Session('1246096278_366122_20230209')
session_folder = session.npexp_path

In [ ]:
session: np_session.Session = np_session.generate_session(mouse, user, session_type='ecephys')
display(session)
session_folder = session.npexp_path
session_folder.mkdir(parents=True, exist_ok=True)

***
## Create platform json

In [8]:
platform_json = np_services.PlatformJsonWriter(path=session_folder)

platform_json.operatorID = user.id
platform_json.mouseID = mouse.id
platform_json.sessionID = session.id

***
## Setup components for the experiment

In [9]:
services = (Sync, VideoMVR, Cam3d, OpenEphys, NewScaleCoordinateRecorder, ScriptCamstim)

***
## Select stimulus session

In [10]:
selected_stim = ttn_widgets.stim_session_select_widget()

***
## Pretest
### Checks before running

In [11]:
np_workflows.check_hardware_widget()

In [12]:
np_workflows.check_openephys_widget()

### Pretest critical components individually
Only proceed if all pretests pass!

In [ ]:
OpenEphys.pretest()

In [ ]:
VideoMVR.pretest()

In [ ]:
Cam3d.pretest()

In [ ]:
Sync.pretest()

In [ ]:
ScriptCamstim.pretest()

***
## Setup and reset all services

In [12]:
ScriptCamstim.script = selected_stim.script #! update after testing
ScriptCamstim.params = selected_stim.params

opto_script: str = selected_stim.opto_script
opto_params: dict = selected_stim.params.get('opto', {})
if opto_params:
    opto_params.update({'mouseID': str(mouse)})

OpenEphys.folder = session_folder.name

NewScaleCoordinateRecorder.log_root = session_folder

for service in services:
    if isinstance(service, Initializable):
        service.initialize()
    if isinstance(service, Testable):
        service.test()

09:43 | ttn_widgets | WARNING | Using hard-coded script in notebooks directory for testing
09:43 | np_services.utils | INFO | sync_device is running on W10DTSM112719
09:43 | np_services.proxies | INFO | Sync initialized: ready for use
09:43 | np_services.proxies | INFO | VideoMVR initialized: ready for use
09:43 | np_services.utils | INFO | camviewer3d is running on W10DTSM112722
09:43 | np_services.proxies | INFO | Cam3d initialized: ready for use
09:44 | np_services.open_ephys | INFO | OpenEphys | Initializing
09:44 | np_services.utils | INFO | open-ephys is running on W10DT713842
09:44 | np_services.open_ephys | INFO | OpenEphys | Testing
09:44 | np_services.open_ephys | INFO | OpenEphys free disk space on \\W10DT713842\A: 1904.0 GB
09:44 | np_services.open_ephys | INFO | OpenEphys free disk space on \\W10DT713842\B: 1810.5 GB
09:44 | np_services.proxies | INFO | NewScaleCoordinateRecorder test passed
09:44 | np_services.proxies | INFO | NewScaleCoordinateRecorder test passed
09:44 

***
## Dip probes

In [3]:
np_workflows.di_widget(platform_json)
np_workflows.photodoc_widget('pre_experiment_surface_image')

NameError: name 'platform_json' is not defined

***
***
# **With mouse on stage**

In [13]:
platform_json.HeadFrameEntryTime = npxc.now()

ERROR! Session/line number was not unique in database. History logging moved to new session 1608


In [8]:
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
np_workflows.photodoc_widget('brain_surface_image')

***
## Probe insertion

In [10]:
platform_json.ProbeInsertionStartTime = npxc.now()
np_workflows.isi_widget(mouse.lims, colormap=False)

OSError: [Errno 22] Invalid argument

***
## Photodoc before advancing probes

In [ ]:
np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer

In [ ]:
np_workflows.print_countdown_timer(minutes=10)

***
## Start devices recording

In [ ]:
for service in (Sync, VideoMVR, OpenEphys):
    service.start()
    if isinstance(service, Verifiable):
        time.sleep(1)
        service.verify()
        
platform_json.ExperimentStartTime = npxc.now()
platform_json.write()

## Start experiment

In [13]:
logger.info('Starting Camstim script')

ScriptCamstim.start()

while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

if isinstance(ScriptCamstim, Finalizable):
    ScriptCamstim.finalize()

logger.info('Camstim script complete')

# opto --------------------------------------------------------------------------------- #
ScriptCamstim.script = opto_script
ScriptCamstim.params = opto_params
logger.info('Starting opto-tagging script')
ScriptCamstim.start()

while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

if isinstance(ScriptCamstim, Finalizable):
    ScriptCamstim.finalize()

logger.info('Opto-tagging script complete')

09:47 | root | INFO | Starting Camstim script
09:47 | np_services.proxies | ERROR | Error accessing ScriptCamstim data path: \\W10DT713938\c$\ProgramData\AIBS_MPE\camstim\data
Traceback (most recent call last):
  File "c:\Users\ben.hardcastle\AppData\Local\pypoetry\Cache\virtualenvs\np-workflows-bIqIfclO-py3.11\Lib\site-packages\np_services\proxies.py", line 518, in ensure_config
    _ = root.exists()
        ^^^^^^^^^^^^^
  File "C:\Users\ben.hardcastle\AppData\Local\Programs\Python\Python311\Lib\pathlib.py", line 1235, in exists
    self.stat()
  File "C:\Users\ben.hardcastle\AppData\Local\Programs\Python\Python311\Lib\pathlib.py", line 1013, in stat
    return os.stat(self, follow_symlinks=follow_symlinks)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
PermissionError: [WinError 5] Access is denied: '\\\\W10DT713938\\c$\\ProgramData\\AIBS_MPE\\camstim\\data'


***
## Stop recording

In [ ]:
while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

for service in (_ for _ in (Sync, VideoMVR, OpenEphys) if isinstance(_, Stoppable)):
    service.stop()

for service in (_ for _ in (VideoMVR, OpenEphys, ScriptCamstim) if isinstance(_, Finalizable)): # Sync will be finalized later to avoid waiting on processing
    service.finalize()

for service in (_ for _ in (VideoMVR, OpenEphys, ScriptCamstim) if isinstance(_, Validatable)):
    service.validate()
    
platform_json.ExperimentCompleteTime = npxc.now()

***
## Before removing probes

In [ ]:
np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()
platform_json.write()

np_workflows.finishing_checks_widget()
np_workflows.copy_files(services, session_folder)